# Лекция 2. Числа

* Представление чисел

# Целые числа (integer)

Компьютер умеет работать с числами в двоичной системе. Но что же это такое? 

Для выбор системы счисления, выбирается основание этой системы. В двоичной - это 2, в шестнадцатиричной - это 16. Чтобы перевести число из любой системы счисления в десятичную, нужно возвести основние системы в степень порядковый номер разряда (считая с 0, а не 1) на число, которое находится в данной позиции. Затем сложить все эти числа.


Пример для двоичной системы
```              
число     |   1   1   0   1   1   0   0   1   1
разряд    |   8   7   6   5   4   3   2   1   0
основание | 2^8 2^7 2^6 2^5 2^4 2^3 2^2 2^1 2^0
значение  | 256+128+  0+ 32+ 16+  0+  0+  2+  1 = 435
```

Пример для шестнадцатиричной, где после 9 идет A(10), B(11), C(12), D(13), E(14), F(15)
```
число     |     1    0    F    A
в 10      |     1    0   15   10
разряд    |     3    2    1    0
основание |  16^3 16^2 16^1 16^0
значение  |  4096+   0+ 240+  10 = 4346
```

* __big-endian__ - старший бит слева, младший справа. Нередко используется в сетевых устройствах. Также используется при обычной записи чисел.
* __little-endian__ - младший бит слева, старший справа. Так храня

In [74]:
def BytesToBin(arr):
    r = ""
    for b in arr:
        r += f'{b:08b} '
    return r

In [77]:
i = 435

print(BytesToBin(i.to_bytes(2, "big", signed=True)))
print(BytesToBin(i.to_bytes(2, "little", signed=True)))

00000001 10110011 
10110011 00000001 


In [78]:
import struct

print(BytesToBin(struct.pack(">h", i)))
print(BytesToBin(struct.pack("<h", i)))

00000001 10110011 
10110011 00000001 


In [81]:
v = "0000000110110011"

r = 0
for p, b in enumerate(v[::-1]):
    r += int(b) * 2**p
r

435

Порой удобно в целых числах кодировать различную информацию. Например, каждый бит может отвечать за различные настройки, а через операци ИЛИ можно объединять настройки.

In [85]:
BOLD = 1 << 0 # 1
RED  = 1 << 1 # 2 
TEXT = 1 << 2 # 4

bin(BOLD | TEXT | RED)

'0b111'

In [61]:
settings = BOLD | TEXT

if settings & BOLD:
    print("BOLD")

BOLD


# Отрицательные числа

Все тоже самое, но первый бит теперь обозначает знак. Если 0 - положительное число, если 1, то отрицательное. Остальные биты инвертируются и к ответу добавляется 1.

```
10001000 => 01110111 => 119 = 120
```

Это очень удобно для арифметики чисел. В этом случае (-1 + 1) и (255 + 1) дадут один и тот же результат для однобайтовой величины.


```
-2 + 5 = 3

-2 +     11111110 +
 5 =     00000101 =
 3    (1)00000011
```

```
254 + 5 = 259 % 256 = 3

254 +     11111110 +
  5 =     00000101 =
  3    (1)00000011
```

In [95]:
bin((-120).to_bytes(1, "big", signed=True)[0])

'0b10001000'

In [98]:
bin((136).to_bytes(1, "big", signed=False)[0])

'0b10001000'

# Переполнение

In [12]:
from ctypes import c_byte, c_ubyte

# c_byte - 1-байтовое целое со знаком
# c_ubyte - 1-байтовое целое без знака

# достигли максимума, идет переполенение
print(c_byte(127 + 1).value)

# достигли минимума
print(c_ubyte(0 - 1).value)

# достигли максимума
print(c_ubyte(255 + 1).value)

-128
255
0


# Числа с плавающей точкой

Вещественные числа обычно записываются в виде

```
[-]мантиса E[+/-] порядок

1.0E+100
```

А кодируются с помощью страшной формулы
$$
(-1)^S * 1,M * 2^{E-127}
$$

### Одинарная точность (32 бита)

<img style="background: white" src='https://fabiensanglard.net/floating_point_visually_explained/floating_point_layout.svg'>
<img src='https://fabiensanglard.net/floating_point_visually_explained/floating_point_math.svg'>

* 1 бит - это бит знака (1 - отрицательное число, 0 - положительное)
* 8 бит - показатель экспоненты (E)
* 23 бита - мантиса (M)

### Двойная точность (64 бита)

В Python используются вещественные числа двойной точности

$$
(-1)^S * 1,M * 2^{E-1023}
$$

* 1 бит - это бит знака (1 - отрицательное число, 0 - положительное)
* 11 бит - показатель экспоненты (E)
* 52 бита - мантиса (M)

Звучит страшно, но показатель экспоненты фактически показывает в каком интервале между соседними степенями двойки находится число, а мантиса показывает в какой из 8388608 ($2^{23}$) или 4503599627370496($2^{52}$) частях этого интервала находится это число. Мантиса - это всегда число 1 и плюс некая вещественная часть. Начиная от старшего бита мы к этой 1 добавляем обратные степени двойки за каждую позицию с битом равным 1 ($b_n * 2^{-(52-n)}$ или $b_n * 2^{-(23-n)}$)

In [34]:
import math
import struct


i = math.e
print(i)

number = BytesToBin(struct.pack(">d", i))
print(number)
number = number.replace(" ", "")

2.718281828459045
01000000 00000101 10111111 00001010 10001011 00010100 01010111 01101001 


In [35]:
S = number[0]
E = number[1:1+11]
M = number[12:]

print(f'S = {S}')
print(f'E = {E}')
print(f'M = {M}')

S = 0
E = 10000000000
M = 0101101111110000101010001011000101000101011101101001


In [99]:
# Вычисляем степень 2-ки

print(E, int(E, base=2), int(E, base=2) - 1023)

10000000000 1024 1


In [100]:
# Считаем мантису

MM = 1.0
for p, b in enumerate(M):
    if b == "1":
        MM += 2**(-p-1)
MM

1.3591409142295225

In [103]:
(-1)**(int(S)) * MM * 2**1

2.718281828459045

# Специальные значения

In [44]:
a = 1.7976931348623157e+308
b = 1.7976931348623157e+308

print(a+b)

number = BytesToBin(struct.pack(">d", a+b))
number

inf


'01111111 11110000 00000000 00000000 00000000 00000000 00000000 00000000 '

In [49]:
a = -1.7976931348623157e+308
b = -1.7976931348623157e+308

print(a+b)

number = BytesToBin(struct.pack(">d", a+b))
number

-inf


'11111111 11110000 00000000 00000000 00000000 00000000 00000000 00000000 '

In [50]:
import numpy as np

number = BytesToBin(struct.pack(">d", np.nan))
number

'01111111 11111000 00000000 00000000 00000000 00000000 00000000 00000000 '

# Домашняя работа

# Задача 2

Имеется `input.txt`, например, со следуюшим содержимым
```
3
AA
FF100
1000
```

В первой строке находится целое число $N$ (в примере - это 3). Далее идет $N$-строк, в каждой строке содержится одно целое число в 16-ричной системе. Нужно преобразовать это число в десятичное, не используя `int(s, base=16)`.

Числа вывести в файл `output.txt`, каждое число в отдельной строке.

Пример `output.txt` для данных выше
```
170
1044736
4096
```